In [132]:
pip install selenium beautifulsoup4 requests

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [133]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import re

In [134]:
driver = webdriver.Chrome()
driver.get("https://www.denvergov.org/Property/")

time.sleep(2)

search_term = "3932 S Wisteria Ct"

search_input = driver.find_element(By.ID, "search")
search_input.send_keys(search_term)

time.sleep(1)
search_input.send_keys(Keys.RETURN)

# Wait for the table to be visible
results_table = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, "results_table"))
)

# Wait for the first link to be clickable
search_result_link = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CLASS_NAME, "search-result-link"))
)
search_result_link = driver.find_element(By.CLASS_NAME, "search-result-link")
# results_table = driver.find_element(By.ID, "results_table")
if search_result_link :
    # search_result_link = results_table.find_element(By.CLASS_NAME, "search-result-link")
    search_result_link.click()
    
    time.sleep(5)
    
else:
    print("No results found")
    
html = driver.page_source

In [135]:
soup1 = BeautifulSoup(html, "html.parser")

property_details = {}



# info_bar_table = soup.find("table", id="property-info-bar")

# if info_bar_table:
    
#     info_bar_headers = info_bar_table.find("thead").find_all("th")
    
#     headers = {}
    
#     for header in info_bar_headers:
#         header.text.strip()
        
#     row_data = []
#     for row in info_bar_table.find("tbody").find_all("tr"):
#         cells = row.find_all("td")
#         for cell in cells:
#             row_data.append(cell.text.strip())
            
#             if len(row_data) == len(headers):
#                 for key, value in zip(headers, row_data):
#                     property_details[key] = value
#             else:
#                 print("lists not same length")
#                 print(row_data)
#                 print(headers)
# else:
#     print("No info bar found")

property_table = soup1.find("table", class_="table-striped")
property_rows = property_table.find_all("tr")
 
for p_row in property_rows:
    cells = p_row.find_all("td")  
    for i in range(0, len(cells), 2):  
        key_raw = cells[i].text.strip()
        pkey = key_raw.replace(":", "") 
        
        if i + 1 < len(cells):
            # value = cells[i + 1].get_text(strip=True)
            value = cells[i + 1].text.strip()
        else:
            value = "Not Found"
        
        
        property_details[pkey] = value
    
    


In [136]:
map_link = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "li#property_map_link > a"))
    )

href_link = map_link.get_attribute("href")


if href_link:
    driver.get(href_link)
    
    # WebDriverWait(driver, 10).until(
    #         EC.presence_of_element_located((By.ID, "property_map")) 
    #     )

else:
    print("No map link found")


WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "summary_map_div")) 
        )
time.sleep(2)
html = driver.page_source

soup2 = BeautifulSoup(html, "html.parser")

In [137]:
map_info_section = soup2.find("div", class_="contentPanel contentContainer")

if map_info_section:
    for item in map_info_section.find_all("div", class_="item"):
        label = item.find("div", class_="label").text.strip()

        value = item.find("div", class_="value")
    
        if value:
            value_text = value.text.strip()
        else:
            value_text = "No value found"

        property_details[label] = value_text
else:
    print("No map info section found")

In [138]:

assessment_link = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "li#Assessment > a"))
    )

href_link = assessment_link.get_attribute("href")


if href_link:
    driver.get(href_link)
else:
    print("No map link found")


WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "assessment_data")) 
        )
time.sleep(2)
html = driver.page_source

soup3 = BeautifulSoup(html, "html.parser")

In [139]:
assessment_details = []

panels = soup3.find_all("div", class_="panel panel-primary")

for panel in panels:
    year = panel.find("h4", class_="panel-title").text.strip()
    table = panel.find("table", class_="table table-striped")
    rows = table.find("tbody").find_all("tr")
    
    
    for row in rows:
        cells = row.find_all("td")
        property_type = cells[0].text.strip()
        actual = cells[1].text.strip()
        assessed = cells[2].text.strip()
        if len(cells) > 3:
            exempt = cells[3].text.strip()
        else:
            exempt = "Not Found"

        assessment_details.append({
            "Year": year,
            "Type": property_type,
            "Actual": actual,
            "Assessed": assessed,
            "Exempt": exempt
        })

# print(assessment_details)
for detail in assessment_details:
    for key, value in detail.items():
        print(f"{key}: {value}")
    print("-" * 20)
    

Year: Current Year
Type: Land
Actual: $224,700
Assessed: $11,370
Exempt: $0
--------------------
Year: Current Year
Type: Improvements
Actual: $589,700
Assessed: $39,510
Exempt: 
--------------------
Year: Current Year
Type: Total
Actual: $814,400
Assessed: $50,880
Exempt: 
--------------------
Year: Prior Year
Type: Land
Actual: $224,700
Assessed: $11,370
Exempt: $0
--------------------
Year: Prior Year
Type: Improvements
Actual: $589,700
Assessed: $39,510
Exempt: 
--------------------
Year: Prior Year
Type: Total
Actual: $814,400
Assessed: $50,880
Exempt: 
--------------------


In [140]:
chain_of_title_link = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "li#Chain_of_Title > a"))
    )

href_link = chain_of_title_link.get_attribute("href")


if href_link:
    driver.get(href_link)
else:
    print("No map link found")


# WebDriverWait(driver, 10).until(
#             EC.presence_of_element_located((By.ID, "assessment_data")) 
#         )
time.sleep(2)
html = driver.page_source

soup4 = BeautifulSoup(html, "html.parser")

In [141]:
chain_of_title_records = []

table = soup4.find("table", class_="table table-striped sortable tablesorter tablesorter-default")
rows = table.find("tbody").find_all("tr")

for row in rows:
    cells = row.find_all("td")
    if len(cells) > 0:
        reception_number = cells[0].text.strip()
        reception_date = cells[1].text.strip()
        instrument = cells[2].text.strip()
        sale_date = cells[3].text.strip()
        sale_price = cells[4].text.strip()
        grantor = cells[5].text.strip()
        grantee = cells[6].text.strip()
        
        chain_of_title_records.append({
            "Reception Number": reception_number,
            "Reception Date": reception_date,
            "Instrument": instrument,
            "Sale Date": sale_date,
            "Sale Price": sale_price,
            "Grantor": grantor,
            "Grantee": grantee
        })
        
for record in chain_of_title_records:
    for key, value in record.items():
        print(f"{key}: {value}")
    print("-" * 20)

Reception Number: 2024019567
Reception Date: 3/8/2024
Instrument: WD
Sale Date: 3/4/2024
Sale Price: $779,000
Grantor: CARR,JOSHUA DANIEL
Grantee: JENSEN,AMANDA
--------------------
Reception Number: 2012067259
Reception Date: 5/23/2012
Instrument: PR
Sale Date: 5/21/2012
Sale Price: $299,000
Grantor: SWIEKATUN,JOHN
Grantee: CARR,JOSHUA DANIEL
--------------------
Reception Number: 2004235773
Reception Date: 11/15/2004
Instrument: DC
Sale Date: 10/2/2004
Sale Price: $0
Grantor: SWIEKATUN,JOHN  & ELAINE
Grantee: SWIEKATUN,JOHN
--------------------


In [142]:
for key, value in property_details.items():
        print(f"{key}: {value}")

Style: 04: TRI-LEVEL W/B
Building Sqr. Foot: 2764
Bedrooms: 5
Baths Full/Half: 3/0
Effective Year Built: 1971
Basement/Finish: 853/423
Lot Size: 14,600
Zoned As: S-SU-F
Mill Levy: 79..202
Document Type: WD
Zoning: Zone District:     S-SU-F       Code Version:       Zoning Map
Neighborhood: Hampden South
Subdivision: HUTCHINSON HILLS FILING NO. 7 WILLOW POINT
Historic Landmark District: No
Individual Historic Landmark: No
Enterprise Zone: Not in enterprise zone.
Floodplain Designation: No value found
Downloadable Maps: Parcel Map Quarter Section Map #SE_143 Assessment Parcel Map Index  Quarter Section Map Index
